In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master/")
import timm
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms as T
from sklearn.metrics import accuracy_score

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

In [ ]:
img = get_img('../input/plants114514/MK/D1/train/Class (1)/R_0Class1 (10).jpg')
plt.imshow(img)
plt.show()

In [ ]:
img = get_img('../input/plants114514/MK/D1/train/Class (10)/R_0Class10 (1).jpg')
plt.imshow(img)
plt.show()

In [ ]:
img = get_img('../input/plants114514/MK/D1/train/Class (11)/R_0Class11 (6).jpg')
plt.imshow(img)
plt.show()

In [ ]:
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm
from pathlib import Path
import warnings 
warnings.filterwarnings('ignore')


class CFG:
    debug=False
    apex=False
    print_freq=100
    num_workers=4
    model_name='resnext50_32x4d'
    size=256
    scheduler='CosineAnnealingWarmRestarts' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=10
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    #T_max=10 # CosineAnnealingLR
    T_0=10 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=32
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    target_size=5
    target_col='label'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    inference=False

if CFG.apex:
    from apex import amp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Utils

In [ ]:

    
def get_transforms(mode):
    
    if mode == 'train':
        return Compose([
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif mode == 'val':
        return Compose([
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    

class PLantDataSet(Dataset):
    
    def __init__(self, images, device, transform=None):
        self.images = images
        self.transform = transform
    
    def __len__(self):
        return self.images.shape[0]
    
    def __getitem__(self, item):
        img = cv2.imread(self.images.iloc[item, 0])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            img = self.transform(image=img)
        label = self.images.iloc[item, 1]
        img = torch.tensor(img["image"]).float().to(device)
        label = torch.tensor(label).long().to(device)
        return img, label

## Models

In [ ]:
class Flatten(nn.Module):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return x.view(x.shape[0], -1)


class VGGNet(nn.Module):
    
    def __init__(self, num_classification, freeze=True):
        super().__init__()
        self.vgg = timm.create_model("vgg16", pretrained=True)
        self.vgg.head = nn.Identity()
        if freeze:
            for param in self.vgg.parameters():
                param.requires_grad = False
        self.classify = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            Flatten(),
            nn.Linear(4096, num_classification))
    
    def forward(self, x):
        x = self.vgg(x)
        return self.classify(x)
    
    
class VGG16Bilinear(nn.Module):
    def __init__(self, num_classification, freeze=True):
        super().__init__()
        vgg = timm.create_model("vgg16", pretrained=True)
        vgg.head = nn.Identity()
        if freeze:
            for param in vgg.parameters():
                param.requires_grad = False
        
        self.cnn = nn.Sequential(vgg, nn.AdaptiveAvgPool2d((1, 1)), nn.Conv2d(4096, 64, 1), nn.ReLU())
        x = torch.rand(1, 3, 256, 256)
        _, c, h, w = self.cnn(x).shape
        
        if freeze:
            for param in self.cnn.parameters():
                param.requires_grad = False
        
        self.classifier = nn.Linear(c ** 2, num_classification)
        
    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.shape
        x = torch.bmm(x.view(b, c, h * w), x.view(b, c, h * w).transpose(1, 2)).view(b, -1)
        x = torch.nn.functional.normalize(torch.sign(x) * torch.sqrt(torch.abs(x) + 1e-10))
        x = self.classifier(x)
        return x
    



## Data Prepare

In [ ]:
from pathlib import Path    
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
train_images = []
d1_train_path = Path("../input/plants114514/MK/D1/train")
for d in d1_train_path.iterdir():
    for f in d.glob("*.jpg"):
        train_images.append((str(f.absolute()), d.name))
train_images = pd.DataFrame(train_images, columns=["path", "label"])

test_images = []
d1_test_path = Path("../input/plants114514/MK/D1/test")
for d in d1_test_path.iterdir():
    for f in d.glob("*.jpg"):
        test_images.append((str(f.absolute()), d.name))
test_images = pd.DataFrame(test_images, columns=["path", "label"])

In [ ]:
label_enc = LabelEncoder().fit(train_images.label)
train_images["label"] = label_enc.transform(train_images["label"])
test_images["label"] = label_enc.transform(test_images["label"])

## Model Train

In [ ]:
def train_epoch(model, optimizer, loss_fn, data_loader):
    losses = []
    model.train()
    for imgs, labels in data_loader:
        preds = model(imgs)
        loss = loss_fn(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)


def eval_epoch(model, loss_fn, data_loader):
    losses = []
    acces = []
    model.eval()
    with torch.no_grad():
        for imgs, labels in data_loader:
            preds = model(imgs)
            loss = loss_fn(preds, labels)
            pred_labels = torch.argmax(preds, dim=1).cpu().numpy()
            labels = labels.cpu().numpy()
            acc = accuracy_score(pred_labels, labels)
            losses.append(loss.item())
            acces.append(acc)
    return np.mean(losses), np.mean(acces)

In [ ]:
device = "cuda"
batch_size = 128
lr = 0.003
num_classifications = 44
train_transform = get_transforms("train")
valid_transform = get_transforms("val")
loss_fn = nn.CrossEntropyLoss()
num_epochs = 3
early_stopping = 5

fold = StratifiedKFold(5)
test_set = PLantDataSet(test_images, device, valid_transform)
test_dl = DataLoader(test_set, batch_size=batch_size, shuffle=False)
for cv, (trn_idx, val_idx) in enumerate(fold.split(train_images.index, train_images.label)):
    
    # model = VGGNet(num_classifications)
    model = VGG16Bilinear(num_classifications)
    model.to(device)
    optimizer = Adam(filter(lambda x: x.requires_grad, model.parameters()), lr)
    best_acc = -float("inf")
    best_acc_epoch = 0
    
    trn = train_images.iloc[trn_idx].reset_index(drop=True)
    val = train_images.iloc[val_idx].reset_index(drop=True)
    trn_set = PLantDataSet(trn, device, train_transform)
    val_set = PLantDataSet(val, device, valid_transform)
    trn_dl = DataLoader(trn_set, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, optimizer, loss_fn, trn_dl)
        val_loss, val_acc = eval_epoch(model, loss_fn, val_dl)
        print(f"CV {cv+1} epoch {epoch} train loss {train_loss:.3f}, val loss {val_loss:.3f} acc {val_acc:.3f}")
        if val_acc > best_acc:
            torch.save(model.state_dict(), f"cv_{cv+1}_best.pth")
    print(f"stop training, load best model epoch {epoch+1}")
    model.load_state_dict(torch.load(f"cv_{cv+1}_best.pth"))
    test_loss, test_acc = eval_epoch(model, loss_fn, test_dl)
    print(f"cv {cv+1} test acc {test_acc:.3f}")